## 카테고리별 베스트 상품 크롤링 자동화

* 패션의류, 신발/잡화, 화장품/헤어, 유아동/출산 .........
* 원하는 만큼 상품 크롤링

#### 자동으로 URL 만들기







In [9]:
# 카테고리별 URL 생성
basic_url = "https://www.11st.co.kr/browsing/BestSeller.tmall?method=getBestSellerMain&cornerNo="

for num in range(1,6):
    url = basic_url + str(num)
    print(url)

https://www.11st.co.kr/browsing/BestSeller.tmall?method=getBestSellerMain&cornerNo=1
https://www.11st.co.kr/browsing/BestSeller.tmall?method=getBestSellerMain&cornerNo=2
https://www.11st.co.kr/browsing/BestSeller.tmall?method=getBestSellerMain&cornerNo=3
https://www.11st.co.kr/browsing/BestSeller.tmall?method=getBestSellerMain&cornerNo=4
https://www.11st.co.kr/browsing/BestSeller.tmall?method=getBestSellerMain&cornerNo=5


#### 자동으로 5개의 카테고리 베스트 상품을 크롤링 하세요

In [187]:
import re
import pandas as pd
import requests
from bs4 import BeautifulSoup
from urllib.request import urlopen

basic_url = "https://www.11st.co.kr/browsing/BestSeller.tmall?method=getBestSellerMain&cornerNo="  # 카테고리별 URL 생성
store_df = pd.DataFrame(columns=['카테고리','상품명','이미지 링크'])                                 # 보기 좋게 표로 정리해서 시각화 하기 위해 판다스 사용

idx = 0 # 인덱스 설정
for num in range(1,6):
    url = basic_url + str(num)
    res = requests.get(url)
    soup = BeautifulSoup(res.content, 'html.parser')

    products = soup.select('#challengeBestAdList > li > div.box_pd > a > div.img_plot > img')   ## 베스트 상품 리스트

    for product in products:
        category = soup.select(f'#metaCtgrLi{str(num)} > button')[0].get_text('type') ## 카테고리명 추출
        category = re.sub(r'[^\w_ ]', '', category)
        product_name = re.sub(r'[^\w_ ]', '', product.get('alt'))                     ## 상품명 추출
        img_url = product.get('src')                                                  ## 이미지 링크 추출
        idx += 1
        store_df.loc[idx] = [category, product_name, img_url]

In [193]:
store_df.head(3)

,카테고리,상품명,이미지 링크
1,신선식품,해남미소 늘해랑 비세척 꿀고구마 10kg,https://cdn.011st.com/11dims/resize/600x600/qu...
2,신선식품,어부백서 국내산 손질 갑오징어 200g팩당 23미10팩 총2kg,https://cdn.011st.com/11dims/resize/600x600/qu...
3,신선식품,찬예찬 프리미엄 명란젓 2kg 젓갈모음,https://cdn.011st.com/11dims/resize/600x600/qu...


In [188]:
store_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 24 entries, 1 to 24
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   카테고리    24 non-null     object
 1   상품명     24 non-null     object
 2   이미지 링크  24 non-null     object
dtypes: object(3)
memory usage: 768.0+ bytes


### 파일 이름이 카테고리별로 정리 되려면 어떻게 해야할까요?
>예) <br>
>img1_겨울 따뜻한 후리스조끼플리스자켓패딩기모바지.jpg <br>
>img1_[달바]화이트 트러플 미스트 세럼 180ml+180ml.jpg <br>
>크롤링한 파일이 섞여있어, 카테고리별 베스트 상품을 파악할 수 없어요. <br>

In [195]:
# 정규표현식을 이용해 파일 이름에 사용할 수 없는 특수문자를 제거
# 카테고리명을 파일 이름에 넣기

for num, df in enumerate(store_df.values):
    ctg_name = df[0]
    prd_name = df[1]
    img_link = df[2]
    
    with urlopen(img_link) as src: #읽어들일 이미지 스트림 열기
        with open(f'./image/{ctg_name}_img{num+1}_{prd_name}.png','wb') as tgt: # w - write b - binary, 쓰기할 이미지 스트림 열기
            img = src.read() # 이미지 읽기
            tgt.write(img)   # 이미지 쓰기기

print("Image Crawling is done")

Image Crawling is done
